In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import os
import plotly.express as px
from datetime import date

In [ ]:
df = pd.read_excel('/Users/kai/Desktop/Weekly report_v0.1.xlsx', None)
df = df['出貨明細']

In [ ]:
today = date.today()
cur_month = today.strftime("%B")
cur_year = int(today.strftime('%Y'))


In [ ]:
test = df.query('BG == "WIP"').groupby(['預交日期', 'Group'])['本國幣別NTD'].sum().reset_index()

In [ ]:
test = test.groupby([pd.Grouper('Group'), pd.Grouper(key='預交日期', freq='Q')])['本國幣別NTD'].sum().reset_index().style.format({'本國幣別NTD':'{0:,.0f}'})

In [ ]:
df = df[(df['預交年份'] == cur_year) & (df['BG'] == 'WIP') & (df['狀態'].str.contains('出'))]

In [ ]:
df1 = df[df['預交月份'] == cur_month]
tree_map = df1.groupby(['BG', '負責業務', 'Group', '品名'])[['本國幣別NTD', '數量']].sum().reset_index()

In [ ]:
fig = px.treemap(tree_map, path=['BG', '負責業務', 'Group', '品名'], values='本國幣別NTD', color='數量')
fig.show()

In [ ]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
               'August', 'September', 'October', 'November', 'December']
df['預交月份'] = pd.Categorical(df['預交月份'], ordered=True, categories=month_order)
fig1 = px.histogram(df, x='預交月份', y='本國幣別NTD', barmode='group', color='Group', category_orders={'預交月份': month_order},
                   labels={'預交月份':'月份', '本國幣別NTD': '營業額', 'Group': '客戶名稱'})
fig1.show()

In [ ]:
with open('WIP月營收.html', 'a') as f:
    f.write(test.render()) # Styler之後要用Render
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig1.to_html(full_html=False, include_plotlyjs='cdn'))
    f.close()